# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Pipeline</span>

**Note**: This tutorial does not support Google Colab.

This is the first part of the quick start series of tutorials about Hopsworks Feature Store. As part of this first module, you will work with data related to credit card transactions. 
The objective of this tutorial is to demonstrate how to work with the **Hopworks Feature Store**  for online data with a goal of training and deploying a model that can predict fraudulent transactions.


## 🗒️ This notebook is divided in 3 sections:
1. Loading the data and feature engineeing,
2. Connect to the Hopsworks Feature Store,
3. Create feature groups and upload them to the Feature Store.

![tutorial-flow](../../../images/01_featuregroups.png)

First of all you will load the data and do some feature engineering on it.

# <span style='color:#ff5f27'> 📝 Imports

In [ ]:
!pip install -U hopsworks --quiet

In [3]:
from math import radians

import numpy as np
import pandas as pd

from features import transactions_fraud

# Mute warnings
import warnings
warnings.filterwarnings("ignore")

First of all you will load the data and do some feature engineering on it.

# <span style='color:#ff5f27'> 📝 Feature Pipeline

## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

The data you will use comes from 2 different CSV files:

- `transactions.csv`: events containing information about when a credit card was used, such as a timestamp, location, and the amount spent. A boolean fraud_label variable (True/False) tells us whether a transaction was fraudulent or not.
- `profiles.csv`: credit card user information such as birthdate and city of residence.

In a production system, these CSV files would originate from separate data sources or tables, and probably separate data pipelines. **These files have a common credit card number column cc_num, which you will use later to join features together from the different datasets.**

Now, you can go ahead and load the data.

In [20]:
# Read the profiles data from a CSV file
profiles_df = pd.read_csv(
    "https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_online/profiles.csv", 
    parse_dates=["birthdate"],
)

# Rename columns for clarity
profiles_df.columns = ["name", "gender", "mail", "birthdate", "City", "Country", "cc_num"]

# Display the first three rows of the DataFrame
profiles_df.head(3)

name gender                      mail  birthdate          City  \
0  Tonya Gregory      F  sandratorres@hotmail.com 1976-01-16  Far Rockaway   
1   Lisa Gilbert      F       michael53@yahoo.com 1986-09-30     Encinitas   
2  Carolyn Meyer      F       anthony47@yahoo.com 2001-07-13        Canton   

  Country            cc_num  
0      US  4796807885357879  
1      US  4529266636192966  
2      US  4922690008243953

In [5]:
# Read the transactions data from a CSV file
trans_df = pd.read_csv(
    "https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_online/transactions.csv", 
    parse_dates=["datetime"],
)

# Display the first three rows of the DataFrame
trans_df.head(3)

tid            datetime            cc_num  \
0  11df919988c134d97bbff2678eb68e22 2022-01-01 00:00:24  4473593503484549   
1  dd0b2d6d4266ccd3bf05bc2ea91cf180 2022-01-01 00:00:56  4272465718946864   
2  e627f5d9a9739833bd52d2da51761fc3 2022-01-01 00:02:32  4104216579248948   

             category  amount  latitude  longitude            city country  \
0       Health/Beauty   62.95  42.30865  -83.48216          Canton      US   
1             Grocery   85.45  33.52253 -117.70755   Laguna Niguel      US   
2  Domestic Transport   21.63  37.60876  -77.37331  Mechanicsville      US   

   fraud_label  
0            0  
1            0  
2            0

In [6]:
# Filter transactions DataFrame to include only rows with category "Cash Withdrawal"
trans_df = trans_df[trans_df.category == "Cash Withdrawal"].reset_index(level=0, drop=True)

# Fill missing values in the 'country' column with "US"
trans_df["country"] = trans_df["country"].fillna("US")

# Filter profiles DataFrame to include only rows with credit card numbers present in the filtered transactions DataFrame
profiles_df = profiles_df[profiles_df.cc_num.isin(trans_df.cc_num.unique())].reset_index(level=0, drop=True)

In [7]:
# Sort the transactions DataFrame by 'datetime' and 'cc_num'
trans_df.sort_values(["datetime", "cc_num"], inplace=True)

---

## <span style="color:#ff5f27;"> 🛠️ Feature Engineering </span>

Fraudulent transactions can differ from regular ones in many different ways. Typical red flags would for instance be a large transaction volume/frequency in the span of a few hours. It could also be the case that elderly people in particular are targeted by fraudsters. To facilitate model learning you will create additional features based on these patterns. In particular, you will create two types of features:
1. **Features that aggregate data from different data sources**. This could for instance be the age of a customer at the time of a transaction, which combines the `birthdate` feature from `profiles.csv` with the `datetime` feature from `transactions.csv`.
2. **Features that aggregate data from multiple time steps**. An example of this could be the transaction frequency of a credit card in the span of a few hours, which is computed using a window function.

Let's start with the first category.

Now you are ready to start by computing the distance between consecutive transactions, lets call it `loc_delta`.
Here you will use the [Haversine distance](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html?highlight=haversine#sklearn.metrics.pairwise.haversine_distances) to quantify the distance between two longitude and latitude coordinates.

In [8]:
# Use the prepare_transactions_fraud function to process the trans_df DataFrame
trans_df = transactions_fraud.prepare_transactions_fraud(trans_df)

# Display the first three rows of the modified DataFrame
trans_df.head(3)

tid            datetime            cc_num  \
0  4c51b54665c7ddb466ea5936f4f3a428 2022-01-01 08:11:01  4467360740682089   
1  4c30185aea2e28e7d9797004710e13c6 2022-01-01 10:03:42  4700702588013561   
2  1a109febabc5c36409f2caf729e110d3 2022-01-01 10:08:59  4205094877256105   

   amount country  fraud_label  loc_delta_t_plus_1  loc_delta_t_minus_1  \
0   77.77      US            0                 0.0             0.000148   
1  781.27      US            0                 0.0             0.000070   
2   36.25      US            0                 0.0             0.000108   

   time_delta_t_minus_1  
0              0.333333  
1              0.416667  
2              0.333333

### <span style="color:#ff5f27;"> 🛠️ On-Demand Transformation Functions </span>

# TODO : Add 
- On-demand transformation function for age-at-tranactions. Add some theory also about the whole thing


---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

### <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

A [feature group](https://docs.hopsworks.ai/3.0/concepts/fs/feature_group/fg_overview/) can be seen as a collection of conceptually related features. In this case, you will create a feature group for the transaction data and a feature group for the windowed aggregations on the transaction data. Both will have `cc_num` as primary key, which will allow you to join them when creating a dataset in the next tutorial.

Feature groups can also be used to define a namespace for features. For instance, in a real-life setting you would likely want to experiment with different window lengths. In that case, you can create feature groups with identical schema for each window length. 

Before you can create a feature group you need to connect to Hopsworks feature store.

In [13]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

2024-09-20 18:57:30,139 INFO: Initializing external client
2024-09-20 18:57:30,139 INFO: Base URL: https://localhost:28181
2024-09-20 18:57:30,969 INFO: Python Engine initialized.

Logged in to project, explore it here https://localhost:28181/p/119


In [14]:
fs.name

'tutorials_featurestore'

To create a feature group you need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group and a version number, if it is not defined it will automatically be incremented to `1`.

In [15]:
# Get or create the 'transactions_fraud_online_fg' feature group
trans_fg = fs.get_or_create_feature_group(
    name="transactions_fraud_online_fg",
    version=1,
    description="Transaction data",
    primary_key=['cc_num'],
    event_time='datetime',
    online_enabled=True,
)

Here you have also set `online_enabled=True`, which enables low latency access to the data. A full list of arguments can be found in the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/feature_store_api/#create_feature_group).

At this point, you have only specified some metadata for the feature group. It does not store any data or even have a schema defined for the data. To make the feature group persistent you need to populate it with its associated data using the `insert` function.

TODO : Add theory that the df should conatin all the required feilds.

In [16]:
# Insert data into feature group
trans_fg.insert(trans_df)
print('✅ Done!')

Feature Group created successfully, explore it at 
https://localhost:28181/p/119/fs/67/fg/22
2024-09-20 18:57:36,525 INFO: 	5 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://localhost:28181/p/119/fs/67/fg/22


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Rows 365112/365112 | Elapsed Time: 00:16 | Remaining Time: 00:00


Launching job: transactions_fraud_online_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://localhost:28181/p/119/jobs/named/transactions_fraud_online_fg_1_offline_fg_materialization/executions
✅ Done!


In [17]:
# Update feature descriptions
feature_descriptions = [
    {"name": "tid", "description": "Transaction id"},
    {"name": "datetime", "description": "Transaction time"},
    {"name": "cc_num", "description": "Number of the credit card performing the transaction"},
    {"name": "amount", "description": "Dollar amount of the transaction"},
    {"name": "country", "description": "Country in which the transaction was made"},
    {"name": "fraud_label", "description": "Whether the transaction was fraudulent or not"},
    {"name": "loc_delta_t_minus_1", "description": "Location of previous transaction"},
    {"name": "time_delta_t_minus_1", "description": "Time of previous transaction"},    
]

for desc in feature_descriptions: 
    trans_fg.update_feature_description(desc["name"], desc["description"])

You can move on and do the same thing for the profile and label feature groups.

In [18]:
# Get or create the 'profile_fraud_online_fg' feature group
profile_fg = fs.get_or_create_feature_group(
    name="profile_fraud_online_fg",
    version=1,
    description="Credit card holder demographic data",
    primary_key=['cc_num'],
    online_enabled=True,
    expectation_suite=expectation_suite_profiles,
)
# Insert data into feature group
profile_fg.insert(profiles_df)
print('✅ Done!')

Feature Group created successfully, explore it at 
https://localhost:28181/p/119/fs/67/fg/23
2024-09-20 18:58:04,769 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://localhost:28181/p/119/fs/67/fg/23


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Rows 1000/1000 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: profile_fraud_online_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://localhost:28181/p/119/jobs/named/profile_fraud_online_fg_1_offline_fg_materialization/executions
✅ Done!


In [19]:
# Update feature descriptions
feature_descriptions = [
    {"name": "cc_num", "description": "Number of the credit card performing the transaction"},
    {"name": "gender", "description": "Gender of the credit card holder"},
]

for desc in feature_descriptions: 
    profile_fg.update_feature_description(desc["name"], desc["description"])

## <span style="color:#ff5f27;">⏭️ **Next:** Part 02 Training Pipeline </span>

In the following notebook you will use our feature groups to create a dataset you can train a model on.